Здесь лежат все нужные функции из блокнота первой части ДЗ + обучается лучшая модель и сохраняются её веса

In [2]:
!pip install scikit-learn==1.7.2 --force-reinstall

  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.7 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully un

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
random.seed(42)
np.random.seed(42)
import math
import re
import phik
from sklearn.model_selection import GridSearchCV
from functools import reduce
from sklearn.linear_model import ElasticNet, Lasso
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error as MSE
from sklearn.linear_model import Ridge
import pickle

# Загрузка данных и очистка

!!! Этот блокнот был нужен, чтобы получить необходимые файлы и не более, и запускался исключительно в Google Colab. Но я его всё равно прикрепила, чтобы было ясно, откуда брались все файлы

In [47]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_test.csv')

print("Train data shape:", df_train.shape)
print("Test data shape: ", df_test.shape)

Train data shape: (6999, 13)
Test data shape:  (1000, 13)


In [48]:
df_train.to_csv('train_data.csv', index=False) # сохранение данных в файлы, чтобы перекинуть их в папку проекта
df_test.to_csv('test_data.csv', index=False)

In [49]:
for i in range(0, len(df_train)):
  try:
    ffil_value = float(df_train.loc[i, 'max_power'][:-4])
    df_train.loc[i, 'max_power'] = ffil_value

  except:
    if df_train.loc[i, 'max_power'] in [np.nan, 'nan']:
      continue
    elif df_train.loc[i, 'max_power'] == '0':
      ffil_value = float(df_train.loc[i, 'max_power'])
      df_train.loc[i, 'max_power'] = ffil_value
    else:
      df_train.loc[i, 'max_power'] = 0


In [50]:
df_train.drop_duplicates(keep='first', subset = [col for col in df_train.columns if col != 'selling_price'], inplace=True)

In [51]:
assert df_train.shape == (5840, 13)

In [52]:
df_train.reset_index(inplace=True, drop=True)

In [53]:
def clean_numeric_col(row):
  cols = {'mileage':['kmpl', 'km/kg'], 'engine': ['CC'], 'max_power': 'bhp'}
  for key in cols.keys():
    if pd.notna(row[key]):
      for pattern in cols[key]:
        row[key] = str(row[key]).replace(pattern, '').strip()
  return row

In [54]:
df_train = df_train.apply(clean_numeric_col, axis=1)

In [55]:
df_train[['mileage', 'engine', 'max_power']] = df_train[['mileage', 'engine', 'max_power']].astype('float')

In [56]:
df_train.dtypes

,0
name,object
year,int64
selling_price,int64
km_driven,int64
fuel,object
seller_type,object
transmission,object
owner,object
mileage,float64
engine,float64


In [57]:
df_test = df_test.apply(clean_numeric_col, axis=1)

In [58]:
df_test[['mileage', 'engine', 'max_power']] = df_test[['mileage', 'engine', 'max_power']].astype('float')

In [59]:
df_train['torque'].sample(5) # быстрый просмотр среза значений

,torque
4582,220Nm at 1400-2600 rpm
177,77Nm@ 3500rpm
167,115Nm@ 3500-3600rpm
3722,215Nm@ 1750-2500rpm
2339,113Nm@ 4500rpm


In [60]:
# так как в данных есть много разных шаблонов и их все тяжело отловить и отследить вручную (я честно пыталась, и потратила на это более часа), код для очистки этого столбца был написан через DeepSeek с передачей ему в качестве контекста всех уникальных значений и описания общей идеи
# далее здесь оставила свои комментарии с пояснениями
def extract_torque_values(torque_str):
    if pd.isna(torque_str):
        return [None, None]

    torque_str = str(torque_str).lower().strip()
    # обработка одного из шаблонов: 400 Nm /2000 rpm
    if '/' in torque_str:
        parts = torque_str.split('/')
        if len(parts) == 2:
            torque_part = parts[0].strip()
            rpm_part = parts[1].strip()
            # извлекаем числа
            torque_match = re.search(r'(\d+\.?\d*)', torque_part)
            rpm_match = re.search(r'(\d+)', rpm_part)
            if torque_match and rpm_match:
                torque = float(torque_match.group(1))
                max_rpm = int(rpm_match.group(1))
                if 'kg' in torque_part:
                    torque *= 9.80665
                return [torque, max_rpm]

    # обработка случаев по шаблону 210 / 1900
    if re.search(r'\d+\s*/\s*\d+', torque_str):
        parts = re.split(r'\s*/\s*', torque_str)
        if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
            return [float(parts[0]), int(parts[1])]

   # ищется шаблон вида: число, пробельный символ (если есть), kgm или nm, пробельный символ (если есть), @ или at, пробельный символ (если есть), число или диапазон, может включать точку, тильду, плюс, дефис,
   # пробельный символ (если есть), rpm (находит его но не включает в результат)
    pattern = r'(\d+\.?\d*)\s*(kgm?|nm?)?\s*(?:@|at)?\s*([\d\s\.,~+\-]+)(?:\s*rpm)'

    match = re.search(pattern, torque_str)

    # в случае, если ничего не найдено
    if not match:
        return [None, None]

    torque_value = float(match.group(1))
    unit = match.group(2) if match.group(2) else ''
    rpm_info = match.group(3).strip()

    if unit and 'kg' in unit:
        torque_value *= 9.80665
    max_rpm = extract_max_rpm(rpm_info)

    return [torque_value, max_rpm]

def extract_max_rpm(rpm_info):
    #очистка от лишних символов
    rpm_info = rpm_info.replace(',', '').replace(' ', '')

    #обработка диапазона - берётся среднее из обеих частей
    if '-' in rpm_info or '~' in rpm_info:
        separator = '-' if '-' in rpm_info else '~'
        parts = rpm_info.split(separator)
        if len(parts) == 2:
            try:
                rpm1 = int(parts[0])
                rpm2 = int(parts[1])
                return (rpm1 + rpm2) // 2
            except ValueError:
                return None

    if '+' in rpm_info:
        base_rpm = rpm_info.replace('+', '')
        if base_rpm.isdigit():
            return int(base_rpm)

    if rpm_info.isdigit():
        return int(rpm_info)
    match = re.search(r'(\d+)', rpm_info)
    if match:
        return int(match.group(1))

    return None

def process_torque_column(df, torque_column='torque'):
    results = df[torque_column].apply(extract_torque_values)
    df['torque'] = results.apply(lambda x: x[0] if x[0] is not None else np.nan)
    df['max_torque_rpm'] = results.apply(lambda x: x[1] if x[1] is not None else np.nan)
    return df

In [61]:
df_train = process_torque_column(df_train)
df_test = process_torque_column(df_test)

In [62]:
for col in df_train.columns:
  if df_train[col].dtype != 'O':
    fill_value = df_train[col].median()
    df_train[col] = df_train[col].fillna(fill_value)
    df_test[col] = df_test[col].fillna(fill_value)

In [63]:
df_train[['engine', 'seats']] = df_train[['engine', 'seats']].astype('int')

In [64]:
df_test[['engine', 'seats']] = df_test[['engine', 'seats']].astype('int')

In [65]:
df_train['name'] = df_train['name'].apply(lambda x: x.lower().split()[0])
df_test['name'] = df_test['name'].apply(lambda x: x.lower().split()[0])

In [66]:
X_train_cat = df_train.drop(columns='selling_price')
X_test_cat = df_test.drop(columns='selling_price')

In [67]:
X_train_cat['seats'] = X_train_cat['seats'].apply(lambda x: str(x))
X_test_cat['seats'] = X_test_cat['seats'].apply(lambda x: str(x))

In [68]:
cat_features = [col for col in X_train_cat.columns if X_train_cat[col].dtype=='O']

In [69]:
cat_features

['name', 'fuel', 'seller_type', 'transmission', 'owner', 'seats']

In [70]:
num_features = [col for col in X_train_cat.columns if X_train_cat[col].dtype!='O' ]

In [71]:
num_features

['year',
 'km_driven',
 'mileage',
 'engine',
 'max_power',
 'torque',
 'max_torque_rpm']

# обучение моделей + их экспорт в Pickle

Здесь были выбраны наилучшая модель и наиболее оптимальная конфигурация стандартизации+кодирования, заново обучены и далее экспортированы в pickle

In [72]:
ridge = Ridge(alpha=1)

In [73]:
y_train_log = np.log(df_train.selling_price)
y_test_log = np.log(df_test.selling_price)

In [74]:
ohe_std_transformer = ColumnTransformer(transformers=[('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), ['name', 'fuel', 'seller_type', 'transmission', 'owner', 'seats']),
                                                  ('standard', StandardScaler(), ['year','km_driven','mileage','engine','max_power','torque','max_torque_rpm'] )], remainder='passthrough', verbose_feature_names_out=False)
ohe_std_transformer.set_output(transform='pandas')

,transformers,"[('ohe', ...), ('standard', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,'first'
,sparse_output,False


In [75]:
ohe_std_transformer.fit(X_train_cat)

,transformers,"[('ohe', ...), ('standard', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,'first'
,sparse_output,False


In [76]:
with open('encoderScaler.pkl', 'wb') as file:
    pickle.dump(ohe_std_transformer, file)
with open('encoderScaler.pkl', 'rb') as file:
    loaded_transformer = pickle.load(file)

In [77]:
X_test_cat

,name,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_torque_rpm
0,mahindra,2010,168000,Diesel,Individual,Manual,First Owner,14.00,2498,112.00,260.00000,7,2000.0
1,tata,2017,25000,Diesel,Individual,Manual,First Owner,21.50,1497,108.50,260.00000,5,2125.0
2,honda,2007,218463,Petrol,Individual,Automatic,First Owner,12.90,1799,130.00,172.00000,5,4300.0
3,honda,2015,173000,Diesel,Individual,Manual,First Owner,25.10,1498,98.60,200.00000,5,1750.0
4,tata,2011,70000,Petrol,Individual,Manual,Second Owner,16.50,1172,65.00,96.00000,5,3000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,hyundai,2008,100000,Petrol,Individual,Manual,Second Owner,19.81,1086,68.05,99.04000,5,4500.0
996,hyundai,2017,50000,Petrol,Individual,Manual,Second Owner,18.60,1197,81.83,114.70000,5,4000.0
997,hyundai,2009,40000,Diesel,Individual,Manual,First Owner,23.00,1396,90.00,219.66896,5,2250.0
998,hyundai,2012,25000,Petrol,Individual,Manual,First Owner,20.36,1197,78.90,111.80000,5,4000.0


In [78]:
X_train_cat

,name,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_torque_rpm
0,maruti,2014,145500,Diesel,Individual,Manual,First Owner,23.40,1248,74.00,190.00000,5,2000.0
1,skoda,2014,120000,Diesel,Individual,Manual,Second Owner,21.14,1498,103.52,250.00000,5,2000.0
2,hyundai,2010,127000,Diesel,Individual,Manual,First Owner,23.00,1396,90.00,219.66896,5,2250.0
3,maruti,2007,120000,Petrol,Individual,Manual,First Owner,16.10,1298,88.20,160.00000,5,2400.0
4,hyundai,2017,45000,Petrol,Individual,Manual,First Owner,20.14,1197,81.86,113.75000,5,4000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5835,hyundai,2008,191000,Petrol,Individual,Manual,First Owner,17.92,1086,62.10,96.10000,5,3000.0
5836,maruti,2013,50000,Petrol,Individual,Manual,Second Owner,18.90,998,67.10,90.00000,5,3500.0
5837,hyundai,2013,110000,Petrol,Individual,Manual,First Owner,18.50,1197,82.85,113.70000,5,4000.0
5838,hyundai,2007,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.80,1493,110.00,160.00000,5,2400.0


In [79]:
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")

scikit-learn version: 1.7.2


In [80]:
X_train_cat = loaded_transformer.transform(X_train_cat)
X_test_cat = loaded_transformer.transform(X_test_cat)

In [81]:
ridge.fit(X_train_cat, y_train_log)

,alpha,1
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [82]:
preds = ridge.predict(X_test_cat)
r2_score(y_test_log, preds)

0.9144461380469331

In [83]:
with open('bestModel.pkl', 'wb') as file:
    pickle.dump(ridge, file)
with open('bestModel.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [84]:
preds = loaded_model.predict(X_test_cat)
r2_score(y_test_log, preds)

0.9144461380469331